In [4]:
!pip install scikit-surprise

In [2]:
import pandas as pd
import numpy as np

from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

In [1]:
!pip uninstall -y numpy
!pip install numpy==1.26.4
!pip install scikit-surprise

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.0 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
pytensor 

In [3]:
data = Dataset.load_builtin('ml-100k')

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [5]:
ratings_data = {
    'User': ['U1', 'U1', 'U2', 'U2', 'U3', 'U3'],
    'Movie': ['M1', 'M2', 'M1', 'M3', 'M2', 'M3'],
    'Rating': [5, 3, 4, 2, 5, 4]
}

df = pd.DataFrame(ratings_data)
df

,User,Movie,Rating
0,U1,M1,5
1,U1,M2,3
2,U2,M1,4
3,U2,M3,2
4,U3,M2,5
5,U3,M3,4


In [6]:
user_item_matrix = df.pivot(index='User', columns='Movie', values='Rating')
user_item_matrix

Movie,M1,M2,M3
User,,,
U1,5.0,3.0,NaN
U2,4.0,NaN,2.0
U3,NaN,5.0,4.0


In [7]:
trainset, testset = train_test_split(data, test_size=0.25)

In [8]:
model = SVD()
model.fit(trainset)

In [9]:
predictions = model.test(testset)

In [10]:
accuracy.rmse(predictions)
accuracy.mae(predictions)

RMSE: 0.9435
MAE:  0.7432


0.7432238034723921

In [11]:
user_id = 10
movie_id = 50

prediction = model.predict(user_id, movie_id)
prediction

Prediction(uid=10, iid=50, r_ui=None, est=3.530213333333333, details={'was_impossible': False})

In [12]:
def get_top_n(predictions, n=5):
    top_n = {}
    for uid, iid, true_r, est, _ in predictions:
        top_n.setdefault(uid, []).append((iid, est))

    for uid in top_n:
        top_n[uid].sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = top_n[uid][:n]
    return top_n

top_recommendations = get_top_n(predictions)

for user, recs in list(top_recommendations.items())[:1]:
    print(f"Top recommendations for User {user}:")
    for movie, rating in recs:
        print(f"Movie {movie} → Predicted Rating {rating:.2f}")

Top recommendations for User 42:
Movie 172 → Predicted Rating 4.64
Movie 64 → Predicted Rating 4.61
Movie 12 → Predicted Rating 4.54
Movie 210 → Predicted Rating 4.54
Movie 357 → Predicted Rating 4.40


Conclusion

Implemented a Collaborative Filtering Recommendation System.Used Matrix Factorization (SVD).Evaluated using RMSE and MAE.Successfully generated personalized recommendations